In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import tensorflow as tf

In [5]:
df_health = pd.read_csv ('healthcare_dataset.csv')

In [6]:
df_health. head ()

,Patient_ID,Age,Gender,Insurance_Type,Chronic_Disease,Mental_Health_Status,Employment_Status,Education_Level,Transportation_Access,Distance_from_Facility,Area_Type,Booking_Date,Appointment_Date,Appointment_Outcome
0,75763,93.0,Male,Medicare,False,True,Student,Advanced Degree,True,37.785038,Suburban,2023-11-04,2024-08-15,Show
1,84945,52.0,Male,Uninsured,False,False,Employed,College,True,NaN,Rural,2023-10-28,2024-07-11,No-Show
2,88559,8.0,Male,Medicare,True,NaN,NaN,Advanced Degree,True,27.277654,Rural,2023-02-12,2024-10-26,Show
3,72217,5.0,Other,Medicare,False,True,Student,College,True,44.618708,NaN,2023-01-30,2024-01-31,Show
4,44071,76.0,Male,Uninsured,True,False,Employed,Advanced Degree,False,12.568497,Urban,2023-11-27,2024-03-03,NaN


In [7]:
df_health = df_health.dropna()

In [8]:
#convert to dates
df_health ['Booking_Date']=pd.to_datetime (df_health['Booking_Date'])
df_health ['Appointment_Date']=pd.to_datetime (df_health['Appointment_Date'])
#difference between in days
df_health['date_difference']=(df_health ['Appointment_Date']-df_health ['Booking_Date'])
df_health['date_difference'] = df_health['date_difference'].dt.days
df_health['Appointment_Outcome'] = df_health['Appointment_Outcome'].replace({'No-Show': 0, 'Show': 1})
df_health.head()
#df_health. info ()


,Patient_ID,Age,Gender,Insurance_Type,Chronic_Disease,Mental_Health_Status,Employment_Status,Education_Level,Transportation_Access,Distance_from_Facility,Area_Type,Booking_Date,Appointment_Date,Appointment_Outcome,date_difference
0,75763,93.0,Male,Medicare,False,True,Student,Advanced Degree,True,37.785038,Suburban,2023-11-04,2024-08-15,1,285
10,94722,76.0,Female,Medicare,True,True,Student,Advanced Degree,False,21.246094,Suburban,2023-02-09,2024-04-06,1,422
11,18645,18.0,Male,Private,False,False,Unemployed,Advanced Degree,True,35.827139,Urban,2023-02-14,2024-09-14,1,578
13,21727,6.0,Male,Private,True,False,Retired,Advanced Degree,True,43.314191,Suburban,2023-12-06,2024-09-12,0,281
20,46269,100.0,Male,Medicaid,False,True,Retired,College,True,30.057164,Urban,2023-11-03,2024-05-05,1,184


In [9]:
#day of the week lets see if something happens
df_health['day_of_week'] = df_health['Appointment_Date'].dt.day_name()
df_health=pd.get_dummies (df_health,dtype=float)
df_health.head ()

,Patient_ID,Age,Distance_from_Facility,Booking_Date,Appointment_Date,Appointment_Outcome,date_difference,Gender_Female,Gender_Male,Gender_Other,...,Area_Type_Rural,Area_Type_Suburban,Area_Type_Urban,day_of_week_Friday,day_of_week_Monday,day_of_week_Saturday,day_of_week_Sunday,day_of_week_Thursday,day_of_week_Tuesday,day_of_week_Wednesday
0,75763,93.0,37.785038,2023-11-04,2024-08-15,1,285,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
10,94722,76.0,21.246094,2023-02-09,2024-04-06,1,422,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
11,18645,18.0,35.827139,2023-02-14,2024-09-14,1,578,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
13,21727,6.0,43.314191,2023-12-06,2024-09-12,0,281,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
20,46269,100.0,30.057164,2023-11-03,2024-05-05,1,184,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [10]:
# Separate features and target variable
X = df_health.drop(['Appointment_Outcome','Appointment_Date','Booking_Date'], axis=1).values
y = df_health['Appointment_Outcome'].values

# X and y
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)



In [11]:
scaler=StandardScaler()
X_scaler=scaler.fit (X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [12]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(units=7, input_dim=X_train_scaled.shape[1], activation='relu'))
model.add(tf.keras.layers.Dense(units=14, activation='relu'))
model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train_scaled, y_train, validation_split=0.15, epochs=100)

# Predictions on the test set
y_pred = (model.predict(X_test_scaled) > 0.5).astype(int)

# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Epoch 1/100
234/234 [==============================] - 0s 654us/step - loss: 0.7066 - accuracy: 0.4961 - val_loss: 0.7023 - val_accuracy: 0.4686
Epoch 2/100
234/234 [==============================] - 0s 436us/step - loss: 0.6950 - accuracy: 0.5157 - val_loss: 0.7011 - val_accuracy: 0.4731
Epoch 3/100
234/234 [==============================] - 0s 422us/step - loss: 0.6927 - accuracy: 0.5256 - val_loss: 0.6995 - val_accuracy: 0.4845
Epoch 4/100
234/234 [==============================] - 0s 428us/step - loss: 0.6916 - accuracy: 0.5251 - val_loss: 0.7008 - val_accuracy: 0.4830
Epoch 5/100
234/234 [==============================] - 0s 409us/step - loss: 0.6906 - accuracy: 0.5293 - val_loss: 0.7000 - val_accuracy: 0.4860
Epoch 6/100
234/234 [==============================] - 0s 421us/step - loss: 0.6899 - accuracy: 0.5333 - val_loss: 0.7002 - val_accuracy: 0.4837
Epoch 7/100
234/234 [==============================] - 0s 408us/step - loss: 0.6891 - accuracy: 0.5349 - val_loss: 0.7002 - val_ac

In [7]:
# Separate features and target variable
X = df_health.drop('Appointment_Outcome', axis=1)
y = df_health['Appointment_Outcome']

# X and y
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# numerical and categorical 
numeric_cols = df_health['Age', 'Distance_from_Facility', 'date_difference']
categorical_features = ['Gender', 'Insurance_Type', 'Mental_Health_Status', 
                         'Employment_Status', 'Education_Level', 'Transportation_Access', 
                         'Area_Type', 'day_of_week']

# Preprocess numerical features
numeric_transformer = StandardScaler()

# Preprocess categorical features
categorical_transformer = ColumnTransformer(
    transformers=[('cat', OneHotEncoder(drop='first'), categorical_features)]
)

# Combine 
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Initialize  within a pipeline
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression())
])

model.fit(X_train, y_train)

#  predictions
y_pred = model.predict(X_test)

# accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.5017035775127768


In [8]:
# Evaluation
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", classification_rep)

Accuracy: 0.5017035775127768
Confusion Matrix:
 [[615 560]
 [610 563]]
Classification Report:
               precision    recall  f1-score   support

     No-Show       0.50      0.52      0.51      1175
        Show       0.50      0.48      0.49      1173

    accuracy                           0.50      2348
   macro avg       0.50      0.50      0.50      2348
weighted avg       0.50      0.50      0.50      2348



In [9]:
print(df_health.columns)


Index(['Patient_ID', 'Age', 'Gender', 'Insurance_Type', 'Chronic_Disease',
       'Mental_Health_Status', 'Employment_Status', 'Education_Level',
       'Transportation_Access', 'Distance_from_Facility', 'Area_Type',
       'Booking_Date', 'Appointment_Date', 'Appointment_Outcome',
       'date_difference', 'day_of_week'],
      dtype='object')


In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

# Example data loading (replace this with your actual data loading)
# df_health = ...

# One-hot encode categorical variables using pd.get_dummies
categorical_columns = ['Gender', 'Insurance_Type', 'Mental_Health_Status', 
                       'Employment_Status', 'Education_Level', 
                       'Transportation_Access', 'Area_Type',]

df_health = pd.get_dummies(df_health, columns=categorical_columns, drop_first=True)

# Separate features and target variable
X = df_health.drop('Appointment_Outcome', axis=1, errors='ignore')
#X = df_health.drop(['Appointment_Outcome', 'day_of_week', 'Appointment_Date'], axis=1)
#X = df_health.drop('Appointment_Outcome', axis=1)
y = df_health['Appointment_Outcome']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Preprocess numerical features
numeric_transformer = StandardScaler()

# Initialize within a pipeline
model = Pipeline(steps=[
    ('scaler', numeric_transformer),
    ('classifier', LogisticRegression())
])

model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

ValueError: could not convert string to float: 'Sunday'